In [3]:
!pip install pandas


  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB 660.6 kB/s eta 0:00:18
   ---------------------------------------- 0.0/11.6 MB 326.8 kB/s eta 0:00:36
   ---------------------------------------- 0.0/11.6 MB 326.8 kB/s eta 0:00:36
   ---------------------------------------- 0.0/11.6 MB 326.8 kB/s eta 0:00:36
   ---------------------------------------- 0.0/11.6 MB 326.8 kB/s eta 0:00:36
   ---------------------------------------- 0.0/11.6 MB 326.8 kB/s eta 0:00:36
   ---------------------------------------- 0.1/11.6 MB 209.5 kB/s eta 0:00:55
   ---------------------------------------- 0.1/11.6 MB 209.5 kB/s eta 0:00:55
   ---------------------------------------- 0.1/11.6 MB 218.3 kB/s eta 0:00:53
   ---------------------------------------- 0.1/11.6 MB 218.3 kB/s eta 0:00:53
   ---------------------------------------- 0.1/11.6 MB 218.3 kB/s eta 0


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\CCL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
!pip install numpy


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\CCL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
!pip install matplotlib

     ---------------------------------------- 0.0/162.3 kB ? eta -:--:--
     ------- -------------------------------- 30.7/162.3 kB ? eta -:--:--
     ------- -------------------------------- 30.7/162.3 kB ? eta -:--:--
     ------- -------------------------------- 30.7/162.3 kB ? eta -:--:--
     --------------------- --------------- 92.2/162.3 kB 435.7 kB/s eta 0:00:01
     --------------------- --------------- 92.2/162.3 kB 435.7 kB/s eta 0:00:01
     --------------------- --------------- 92.2/162.3 kB 435.7 kB/s eta 0:00:01
     ------------------------ ----------- 112.6/162.3 kB 327.2 kB/s eta 0:00:01
     ------------------------------------ 162.3/162.3 kB 405.5 kB/s eta 0:00:00
  Using cached pyparsing-3.1.1-py3-none-any.whl.metadata (5.1 kB)
   ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.6 MB 1.9 MB/s eta 0:00:04
   ---------------------------------------- 0.0/7.6 MB 1.9 MB/s eta 0:00:04
    --------------


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\CCL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import certifi

os.environ['SSL_CERT_FILE'] = certifi.where()


df= pd.read_csv('ration-reserve-id.csv')

def normalize_data(original_data):
    data_min = np.min(original_data, axis=0)
    data_max = np.max(original_data, axis=0)
    data_range = data_max - data_min
    data_normalized =  (original_data - data_min) / data_range
    return data_normalized

#Drop the y label from dataframe
features = df.drop(['lifetimeRepayments_7day_forecast'], axis=1)
#setting our y label
target = df['lifetimeRepayments_7day_forecast']
# convert data to numpy format
X_original = features.to_numpy()
Y_original = target.to_numpy()
# normalize the data 
X_normalized= normalize_data(X_original)
y_normalized= normalize_data(Y_original)
df_forecast = df[-7:]
df_forecast_data = df_forecast.to_numpy()

# Convert the original data to Cairo 
def generate_cairo_files(data, name, folder_name):
    os.makedirs(f'multiple_linear_regression/src/datasets/{folder_name}', exist_ok=True)
    with open(os.path.join('multiple_linear_regression/src/datasets', f'{folder_name}', f"{name}.cairo"), "w") as f:
            f.write(
                "use array::ArrayTrait;\n" +
                "use orion::numbers::fixed_point::implementations::fp16x16::core::{FP16x16Impl, FP16x16PartialEq };\n" +
                "use orion::operators::tensor::{Tensor, TensorTrait, FP16x16Tensor};\n" +
                "use orion::numbers::{FP16x16, FixedTrait};\n\n" +
                "fn {0}() ->  Tensor<FP16x16>  ".format(name) + "{\n" +
                "    let tensor = TensorTrait::<FP16x16>::new( \n"
            )
            if len(data.shape)>1:
                f.write("    shape: array![{0},".format(data.shape[0]))
                f.write("{0}].span(),\n".format(data.shape[1]))
                f.write(
                    "    data: array![ \n"
                )
            if len(data.shape)==1:
                f.write("    shape: array![{0}].span(),\n".format(data.shape[0]))
                f.write(
                    "    data: array![ \n"
                )
            for val in np.nditer(data.flatten()):
                f.write("    FixedTrait::new({0}, {1} ),\n".format(abs(int(val * 2**16)), str(val < 0).lower()))
            f.write(
                "].span() \n \n" +
                ");\n\n"+
                "return tensor; \n"+
                "}"
            )
    with open(os.path.join('multiple_linear_regression/src/datasets', f'{folder_name}.cairo'), 'a') as f:
        f.write(f"mod {name};\n")

generate_cairo_files(X_original, 'aave_x_features', 'aave_data')
generate_cairo_files(Y_original, 'aave_y_labels', 'aave_data')
generate_cairo_files(df_forecast_data, 'aave_weth_revenue_data_input', 'user_inputs_data')
